We got 16 pages of data in one scrape. We'll go ahead with this as our main dataframe. If necessary we can add another 15 pages of data, for a total of around 3,000 datapoints. 

In [1]:
import pandas as pd
import numpy as np
import bs4
import requests
import neweggutils
import re

from collections import OrderedDict

%load_ext autoreload
%autoreload 2

In [3]:
!ls

1-5_master.csv                      extract-clean-newegg.ipynb
1-5comp.csv                         first_5_components.csv
1-5price.csv                        first_5_components_backup.csv
11_to_14_components.csv             first_5_prices.csv
11_to_14_prices.csv                 first_5_prices_backup.csv
16_comp.csv                         newegg-utils.py
16_price.csv                        neweggutils.py
6-11comp.csv                        prices.csv
6-11price.csv                       scale-scrape.ipynb
6_to_11_component.csv               scrape-amazon.ipynb
6_to_11_components.csv              scrape-newegg.ipynb
6_to_11_price.csv                   scrape-nutrition.ipynb
6_to_11_prices.csv                  scrape-stocks.ipynb
ETL-clean.ipynb                     test_components.csv
README.md                           test_components.html
__pycache__                         test_prices.csv
components.csv                      tibbott-project-luther-proposal.pdf


In [6]:
first5 = pd.read_csv('1-5_master.csv', index_col=0)

In [ ]:
def explode_dfs(components, prices):
    '''
    
    '''
    components.drop(labels=['Unnamed: 0', 'price_html'], axis=1, inplace=True)
    prices.drop(labels=['Unnamed: 0', 'components_html'], axis=1, inplace=True)
    
    components = components.apply(lambda x: neweggutils.get_components(x[0]), axis=1)
    prices = prices.apply(lambda x: neweggutils.get_prices_and_links(x[0]), axis=1)
    
    components = components.dropna()
    prices = prices.dropna()
    
    components = pd.DataFrame.from_records(components.values)
    prices = pd.DataFrame.from_records(prices.values)
    

Load pages 16 and on dataframe:

In [75]:
sixteen_c = pd.read_csv('16_price.csv')
sixteen_p = pd.read_csv('16_comp.csv')

Drop unnecessary columns.

In [76]:
sixteen_c.drop(labels=['Unnamed: 0', 'price_html'], axis=1, inplace=True)
sixteen_p.drop(labels=['Unnamed: 0', 'component_html'], axis=1, inplace=True)

Parse HTML to get price and corresponding links from `sixteen_p`, and price and corresponding components from `sixteen_c`. We will use the links to join corresponding prices and components.

In [ ]:
sixteen_c = sixteen_c.apply(lambda x: neweggutils.get_components(x[0]), axis=1)
sixteen_p = sixteen_p.apply(lambda x: neweggutils.get_prices_and_links(x[0]), axis=1)

In [63]:
sixteen_c = sixteen_c.dropna()

In [53]:
x = sixteen_c.values

In [66]:
components = pd.DataFrame.from_records(sixteen_c.values)

In [71]:
sixteen_p = sixteen_p[:, np.newaxis]

In [74]:
sixteen_p.shape

(15, 1)